# 7.1 - Macrobond Chart Server - Rendering a chart as an image

*How to use Macrobond Chart Server functionality to render charts extracted from documents*

This notebook aims to provide examples of how to use Macrobond's Chart Server API to render a chart defined in a Macrobond document as an image.

We will focus here on using the Macrobond Chart Library which is a public folder any Macrobond's subscriber can access. It contains, among other things, the Chart of the Week sub-directory.

Please, refer to our technical documentation to install Chart Server and insert the credentials provided by your Macrobond's representative:
https://help.macrobond.com/technical-information/the-macrobond-chart-server/

*Full error handling is omitted for brevity*

`Note that this notebook is using a Macrobond-setup internal server: https://appt.macrobondfinancial.com. This notebook will not work on the client's side as plug-and-play. Client will need to setup their own server communicating with Macrobond's URLs. Please, refer to the above documentation for more information.`

***

## Importing packages

This notebook requires Python Image Library - pillow, please run pip install pillow if you don't have it installed already.

In [0]:
import json
import pandas as pd
from io import BytesIO
from PIL import Image
from urllib.request import urlopen

pd.options.display.max_colwidth = 1000

***

## Listing the directories

Let's first retrieve the list of directories where documents are stored.
Note that in the examples we are using an internal server. You should replace the root URL with the one for your Chart Server as per the documentation: https://help.macrobond.com/technical-information/the-macrobond-chart-server/ under section 'Addresses of Macrobond servers' after you have successfully authenticated.

The 'listdirs' returns a list of folders present in a specified directory. If no parameter is specified, the root folders will be listed. This function, combined with the 'listfiles', can be used to implement user interface for browsing the directories. Returned output is in JSON format.

In [0]:
json_url = urlopen("https://renderdemointernal.mbnd.eu/chartserver/listdirs?")
data = json.loads(json_url.read())

df = pd.DataFrame(data)
df

In the examples below, we are use the path for the Macrobond Chart Library. This is the repository where you can find the 'Charts of the Week' Macrobond publishes every Friday.

In [0]:
json_url = urlopen(
    "https://renderdemointernal.mbnd.eu/chartserver/listdirs?path=macrobond://macrobondfilestore/macrobond/filestore/"
)
data = json.loads(json_url.read())

df = pd.DataFrame(data)
df

Sub-directories can be listed for directories where HasChildren is true by calling listdirs with the specified Path. 

Let's now focus on the Charts of the Week directory and use the Path found in the dataframe above into the query.

In [0]:
json_url = urlopen(
    "https://renderdemointernal.mbnd.eu/chartserver/listdirs?path=macrobond://macrobondfilestore/macrobond/filestore/Charts%20of%20the%20week/"
)
data = json.loads(json_url.read())

df = pd.DataFrame(data)
df

In [0]:
json_url = urlopen(
    "https://renderdemointernal.mbnd.eu/chartserver/listdirs?path=macrobond://macrobondfilestore/macrobond/filestore/Charts%20of%20the%20week/2023/"
)
data = json.loads(json_url.read())

df = pd.DataFrame(data)
df

In [0]:
json_url = urlopen(
    "https://renderdemointernal.mbnd.eu/chartserver/listdirs?path=macrobond://remotepublicfilestore/macrobond/filestore/Charts%20of%20the%20week/2023/01%20January/"
)
data = json.loads(json_url.read())

df = pd.DataFrame(data)
df

Now that HasChildren is not true, then this directory can contain files and the path can be passed to listfiles in order to list the files. 

Let's list the available documents we can render.

***

## List files
The `listfiles` returns files located in specified directory. Output is in JSON format. 

In [0]:
json_url = urlopen(
    "https://renderdemointernal.mbnd.eu/chartserver/listfiles?path=macrobond://macrobondfilestore/macrobond/filestore/Charts%20of%20the%20week/2023/01%20January/13th/&ext=.mbnd "
)
data = json.loads(json_url.read())

df2 = pd.DataFrame(data)
df2

Next, lets check the detailed info of a document that we want to render

***

## Document Information

The `documentinfo` call returns all the available charts (presentations) of the document you have selected, along with the metadata of each chart. 

In [0]:
json_url = urlopen(
    "https://renderdemointernal.mbnd.eu/chartserver//documentinfo?path=macrobond://macrobondfilestore/macrobond/filestore/Charts%20of%20the%20week/2023/01%20January/13th/US%20homesales%20tightening.mbnd"
)
data = json.loads(json_url.read())
pd.DataFrame(data)

We can now select which document we wish to render.

***

## Render a document
The `renderpath` operation renders a Macrobond document posted as XML. 


### Render Charts

You can decide which chart to be rendered should your document contain multiple charts, by specifying the presentation id in the document info.

For example, the two charts below are fetched from the same document which path is in the last section. 


We sepecify the id of the wanted chart with parameter **presentation**.

In [0]:
chart = urlopen(
    "https://renderdemointernal.mbnd.eu/chartserver/renderpath?path=macrobond://remotepublicfilestore/macrobond/filestore/Charts%20of%20the%20week/2023/01%20January/13th/US%20homesales%20tightening.mbnd&presentation=9ba2cf6e-9767-4d37-ad87-8300e360955f&width=800&height=600&format=png&dpi=96"
)
img = Image.open(BytesIO(chart.read()))
img

### Render Animated Charts

This endpoint can also render the chart as an animated svg file (requires Chart Server version 1.28 or later), 
* format=svg
* useAnimation=true

Optional:
* defaultAnimationDuration: set the duration of your animated chart, in seconds
* defaultAnimationDelay: set the delay of your animation, in seconds

In [0]:
from IPython.display import HTML

chart = urlopen(
    "https://renderdemointernal.mbnd.eu/chartserver/renderpath?path=macrobond://remotepublicfilestore/macrobond/filestore/Charts%20of%20the%20week/2023/01%20January/13th/US%20homesales%20tightening.mbnd&presentation=9ba2cf6e-9767-4d37-ad87-8300e360955f&width=800&height=600&format=svg&dpi=96&useAnimation=true&defaultAnimationDuration=2&defaultAnimationDelay=0.5"
)

display(HTML(chart.read().decode("utf-8")))

***

## Search files
The `searchfiles` returns list of files containing given search-string, located in specified directory and its direct and indirect sub-folders. Output is in JSON format. 


In the example below, we will look for a chart contained in a document called **'Los Angeles - United States Port traffic'**, located into the Macrobond's Chart Library.
As per the **Listing the directories** section in this notebook, we know that the path for this folder is: macrobond://macrobondfilestore/macrobond.

If you wanted to look for a chart located in your Company directory, you would use path = macrobond://remotepublicfilestore/common/filestore/	

**Remember to use UTF-8 encoding for the charts you are looking for in the &query part of the URL**

In [0]:
json_url = urlopen(
    "https://renderdemointernal.mbnd.eu/chartserver/searchfiles?path=macrobond://macrobondfilestore/macrobond/&skipThumbnails=true&ext=.mbnd&query=Los%20Angeles%20-%20United%20States%20Port%20traffic"
)
data = json.loads(json_url.read())

df3 = pd.DataFrame(data)
df3

You can now use the Path returned by the previous response to render the chart contained in this document, as explained in section **'Render a document'**.

In [0]:
chart2 = urlopen(
    "https://renderdemointernal.mbnd.eu/chartserver/renderpath?path=macrobond://macrobondfilestore/macrobond/filestore/Charts%20of%20the%20week/2021/October/Oct%2022nd/Los%20Angeles%20-%20United%20States%20Port%20traffic.mbnd&width=800&height=600&format=png&dpi=96"
)
img2 = Image.open(BytesIO(chart2.read()))
img2

***

### Document Metadata

The `filemeta` endpoint returns several metadata attributes of the document you have selected, including:

1. **Timeseries Name**: primname of the timeseries used in the document.

2. **Primary Chart Comment**: The comment associated with the primary chart of the document. If the document has multiple charts, each chart's comment can be found using the `documentinfo` endpoint.

3. **Account Information**:
   - **Created By**: the Macrobond user account who created the document
   - **Last Saved By**: the Macrobond user account who last saved the document

For more information on the document's creation and last modification time stamps, please refer to the `listfiles` or `searchfiles` endpoints.

In the example below, we will only display "SeriesNames" which contains the raw timeseries alias called into the document and "SeriesPrimaryNames" which returns the underlying Macrobond primary name of the time series used. Note that this function excludes the potential calculations you may have used when creating your document.

In [0]:
json_url = urlopen(
    "https://renderdemointernal.mbnd.eu/chartserver/filemeta?path=macrobond://macrobondfilestore/macrobond/filestore/Charts%20of%20the%20week/2023/01%20January/13th/US%20GDP%20Nowcast.mbnd"
)
metadata = json.loads(json_url.read())
metadata_df = pd.DataFrame(metadata)
metadata_df[["SeriesNames", "SeriesPrimaryNames"]]

If our document's primary chart contains comments, the comment is in format of UTF8 accepting multi-line text, including XML/JSON.

You can call this section below:

In [0]:
metadata["Comment"]

***

### Document Data

Apart from the metadata above, for users with license of Chart Server with data access, the `calculatepath` endpoint allows you to retrieve the data of underlying chart, including:

1. **title**: the chart's title and its font details

2. **xAxis**: the x-axis' data, including the values on x-axis and its font details, etc.

3. **panes**: details of the panes, including the y-axis' data and graph's values

4. And other chart related setting details such as border, background, etc.

In [0]:
json_url = urlopen(
    "http://localhost:8088/chartserver/calculatepath?path=macrobond://remotepublicfilestore/macrobond/filestore/Examples/Analyses/Aggregate/Aggregate%20by%20fiscal%20year.mbnd"
)
metadata = json.loads(json_url.read())
metadata.keys()

If one document contains multiple charts, you need to specify the presentation id in the URL to retrieve the data of the chart you want.

In [0]:
json_url = urlopen(
    "https://renderdemointernal.mbnd.eu/chartserver//documentinfo?path=macrobond://remotepublicfilestore/library/filestore/Macro%20Themes/1.%20United%20States/Trade%20Balance%20%26%20BoP/US-CurrentAccount3.mbnd"
)
data = json.loads(json_url.read())
pd.DataFrame(data)

In [0]:
json_url = urlopen(
    "http://localhost:8088/chartserver/calculatepath?path=macrobond://remotepublicfilestore/library/filestore/Macro%20Themes/1.%20United%20States/Trade%20Balance%20%26%20BoP/US-CurrentAccount3.mbnd&presentation=a474f4a3-f45d-4e66-98fb-8357725ebf8a"
)
metadata = json.loads(json_url.read())
metadata["title"]

You can use the data in charts to rebuild the chart as you wish, or use it for further analysis.

In [0]:
pd.DataFrame(
    {
        "dates": [pd.to_datetime(i) for i in metadata["xAxis"]["observationDates"]],
        "value": metadata["panes"][0]["graphGroups"][0]["graphs"][0]["observations"],
    }
).plot(x="dates", y="value", title=metadata["title"]["text"])